In [45]:
import os
import polars as pl
from deltalake import DeltaTable, write_deltalake
from typing import List

def SaveDataFrameAsDeltaLake(
    df: pl.DataFrame,
    delta_dir: str,
    target_mb_per_partition: int = 512,
    sample_rows: int = 10,
    buffer_rows: int = 50_000
) -> str:
    """
    Saves a Polars DataFrame to a Delta Lake directory using chunked streaming.

    - Estimates row size by writing first `sample_rows` to Parquet.
    - Computes rows_per_partition based on target partition size.
    - Writes chunk-by-chunk (overwrite → append).
    - Never loads/creates new big frames except the current chunk.
    """

    if df.is_empty():
        raise ValueError("Input DataFrame is empty")

    os.makedirs(delta_dir, exist_ok=True)

    # ----------------------------------------
    # 1. Estimate row size
    # ----------------------------------------
    n = min(sample_rows, len(df))
    df_sample = df.head(n)

    tmp_path = "tmp_row_sample.parquet"
    df_sample.write_parquet(tmp_path)
    row_size = os.path.getsize(tmp_path) / n
    os.remove(tmp_path)

    # Compute rows per partition
    rows_per_partition = max(
        int((target_mb_per_partition * 1024 * 1024) / row_size),
        buffer_rows
    )

    print(f"Target rows per partition ≈ {rows_per_partition:,}")

    # ----------------------------------------
    # 2. Chunked write to Delta Lake
    # ----------------------------------------
    first_write = True
    total_rows = len(df)
    chunk_idx = 1

    for start in range(0, total_rows, rows_per_partition):

        end = min(start + rows_per_partition, total_rows)
        print(f"Writing rows {start:,} → {end-1:,}")

        df_chunk = df.slice(start, end - start)

        write_deltalake(
            delta_dir,
            df_chunk,
            mode="overwrite" if first_write else "append",
            schema_mode="merge"
        )
        first_write = False

        print(f"→ Written partition #{chunk_idx} with {len(df_chunk):,} rows")
        chunk_idx += 1

    print("\n==========================================")
    print(" DATAFRAME → DELTA COMPLETED 🎉")
    print(" Delta path:", delta_dir)
    print("==========================================\n")

    return delta_dir

from enum import Enum
class DataType(Enum):
    """Enumeration for data types."""
    Unknown = 0
    Numeric = 1
    DateTime = 2
    Float = 3
    String = 4
    Formula = 5
    Blank = 6
    Boolean = 7
    Error = 8


def TransformProvidedSourceDataOfOxyzo(delta_folder_path: str) -> pl.DataFrame:
    """
    Transforms the wide Oxyzo Delta Lake data into a long format.  
    It dynamically finds the EOM header row, handles N blocks, and uses Polars 'unpivot'. 
    Now with prefix matching for metrics. 
    """
    # 1. Load Delta Lake
    dt = DeltaTable(delta_folder_path)
    df = pl.from_arrow(dt. to_pyarrow_table())

    print("Loaded DF")
    print(df)

    # 2. Define Constants
    FIXED_COL_COUNT = 5
    METRIC_BLOCK_SIZE = 7
    
    fixed_cols = ["Loan ID", "Loan No.", "Org ID", "Org Name", "Sanction Date"]
    metrics = [
        "Current max DPD", "Lifetime max DPD", "Lifetime max DPD on", 
        "Total Overdue", "Principal Overdue", "EMI Overdue", "POS"
    ]

    column_type_map = {
        "Loan ID": DataType.String,
        "Loan No.": DataType.String,
        "Org ID": DataType.String,
        "Org Name": DataType.String,
        "Sanction Date": DataType.DateTime,
        "EOM": DataType.DateTime,
        "Current max DPD": DataType.Float,
        "Lifetime max DPD": DataType.Float,
        "Lifetime max DPD on": DataType.DateTime,
        "Total Overdue": DataType.Float,
        "Principal Overdue": DataType.Float,
        "EMI Overdue": DataType.Float,
        "POS": DataType.Float
    }

    SortedDataFrameColumnName = "Org ID"

    # 3. Dynamic Date Row Detection (The Core Fix)
    date_row_index = -1
    eom_row = None
    
    for row_idx in range(min(5, df.height)):
        current_row = df.row(row_idx)
        
        for check_idx in range(5, 8):
            val = current_row[check_idx]
            if isinstance(val, str) and val.strip() and '-' in val:
                eom_row = current_row
                date_row_index = row_idx
                break
        
        if eom_row is not None:
            break
    
    if eom_row is None:
        raise ValueError("❌ Critical Error: Could not locate the EOM date header row within the first 5 rows of the Delta Lake data.")

    # 4.  Calculate N (Number of EOM blocks)
    total_cols = df.width
    
    if (total_cols - FIXED_COL_COUNT) < 0 or (total_cols - FIXED_COL_COUNT) % METRIC_BLOCK_SIZE != 0:
        raise ValueError(f"❌ Invalid column count ({total_cols}). Expected 5 fixed columns plus N * 7 metric columns.")
    
    n_blocks = (total_cols - FIXED_COL_COUNT) // METRIC_BLOCK_SIZE
    
    # 5. Extract EOM Dates resiliently from the found row
    eom_dates: List[str] = []
    
    for i in range(n_blocks):
        start_index = FIXED_COL_COUNT + (i * METRIC_BLOCK_SIZE)
        end_index = start_index + METRIC_BLOCK_SIZE
        
        block_slice = eom_row[start_index : end_index]
        date_val = None
        
        for val in block_slice:
            if isinstance(val, str) and val.strip() and '-' in val:
                date_val = val.strip()
                break
        
        if date_val is None:
             raise ValueError(f"❌ Date found the date row (index {date_row_index}), but still failed to find date in block {i+1}. Block slice: {block_slice}")
            
        eom_dates.append(date_val)

    # 6. Generate New Column Names
    new_columns = fixed_cols. copy()
    for date in eom_dates:
        for metric in metrics:
            new_columns.append(f"{date}||{metric}")

    # 7. Apply Names and Slice Data
    df_data = df.slice(date_row_index + 1)
    df_data = df_data.select(df_data.columns[:len(new_columns)])
    df_data.columns = new_columns

    # 8.  Unpivot (formerly melt) - Transform Wide to Long
    df_long_vars = [c for c in new_columns if "||" in c]

    df_unpivoted = df_data.unpivot(
        index=fixed_cols,
        on=df_long_vars,
        variable_name="EOM_Metric",
        value_name="Value"
    )

    # 9. Split EOM and Metric and Pivot Back (Standard procedure)
    df_long = df_unpivoted.with_columns(
        pl.col("EOM_Metric").str.split_exact("||", 1)
        .struct.rename_fields(["EOM", "Metric"])
        .alias("temp")
    ).unnest("temp").drop("EOM_Metric")

    df_final = df_long.pivot(
        on="Metric",
        index=fixed_cols + ["EOM"],
        values="Value",
        aggregate_function="first",
        sort_columns=True
    )

    # 10. Type Casting with Prefix Matching
    cols_to_cast = []
    final_order = fixed_cols + ["EOM"] + metrics
    
    # Helper function for prefix matching
    def get_matching_metric(col_name: str, metrics_list: List[str]) -> str:
        """Find the best prefix match for a column name from the metrics list."""
        for metric in metrics_list:
            if col_name.startswith(metric) or metric in col_name:
                return metric
        return col_name  # Return original if no match found
    

    # Casting to datatype
    def parse_date_column(col_name):
        """
        Parse a date column with multiple format attempts and fallbacks.
        """
        col = pl.col(col_name)
        
        # Try multiple datetime formats in sequence
        parsed = (
            col.str.strptime(pl. Datetime, "%Y-%m-%d %H:%M:%S %Z", strict=False)
            .fill_null(col.str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%d-%m-%Y %H:%M:%S", strict=False))
            .fill_null(col.str.strptime(pl. Datetime, "%m-%d-%Y %H:%M:%S", strict=False))
            .fill_null(col.str.strptime(pl. Datetime, "%Y-%m-%d", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%d-%m-%Y", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%m-%d-%Y", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%d/%m/%Y %H:%M:%S", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%m/%d/%Y %H:%M:%S", strict=False))
            .fill_null(col. str.strptime(pl. Datetime, "%d/%m/%Y", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%m/%d/%Y", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%Y/%m/%d", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%d. %m.%Y %H:%M:%S", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%d.%m.%Y", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%Y.%m.%d", strict=False))
            # ISO 8601 formats
            .fill_null(col.str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S", strict=False))
            .fill_null(col.str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%SZ", strict=False))
            .fill_null(col.str.strptime(pl. Datetime, "%Y-%m-%dT%H:%M:%S%. fZ", strict=False))
            # Unix timestamp (seconds)
            .fill_null(col.cast(pl.Float64, strict=False). cast(pl.Datetime("us"), strict=False))
        )
        
        # Extract just the date part
        return parsed. dt.date(). alias(col_name)

    for actual_col in df_final.columns:
        if actual_col in column_type_map:
            dtype = column_type_map[actual_col]

            if dtype == DataType.DateTime:
                cols_to_cast.append(parse_date_column(actual_col))
            elif dtype in (DataType.Float, DataType.Numeric):
                cols_to_cast.append(
                    pl.col(actual_col).cast(pl.Float64, strict=False).alias(actual_col)
                )
            elif dtype == DataType.String:
                cols_to_cast.append(
                    pl.col(actual_col).cast(pl.Utf8, strict=False).alias(actual_col)
                )
            elif dtype == DataType.Boolean:
                cols_to_cast.append(
                    pl.col(actual_col).cast(pl.Boolean, strict=False).alias(actual_col)
                )
            else:
                # Fallback: cast to string
                cols_to_cast.append(
                    pl.col(actual_col).cast(pl.Utf8, strict=False).alias(actual_col)
                )

    df_final = df_final.with_columns(cols_to_cast)

    df_final = df_final.select(final_order).sort(SortedDataFrameColumnName)

    return df_final.select(final_order)

In [46]:

pl.Config.set_fmt_str_lengths(2000)   # widen column display
pl.Config.set_tbl_width_chars(2000)   # expand table width
pl.Config.set_tbl_rows(50000)         # show up to 50k rows
pl.Config.set_tbl_cols(500) 

def main():
    result = TransformProvidedSourceDataOfOxyzo("E:\\Dev\\Oxyzo R&D\\RndGit\\SourceDeltaLake")
    print("Transformed DataFrame:"  )
    print(result)

    os.makedirs("E:\\Dev\\Oxyzo R&D\\RndGit\\Output", exist_ok=True)
    result.write_csv("E:\\Dev\\Oxyzo R&D\\RndGit\\Output\\TransformedDeltaLake_CSV_Output.csv")
    

    SaveDataFrameAsDeltaLake(
        result,
        delta_dir="E:\\Dev\\Oxyzo R&D\\RndGit\\Output\\TransformedDeltaLake"
    )

if __name__ == "__main__":
    main()

Loaded DF
shape: (5, 19)
┌─────────┬─────────────┬────────┬──────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬───────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│ Loan ID ┆ Loan No.    ┆ Org ID ┆ Org Name ┆ Sanction Date       ┆ Current max DPD     ┆ Lifetime max DPD    ┆ Lifetime max DPD on ┆ Total Overdue       ┆ Principal Overdue   ┆ EMI Overdue         ┆ POS                 ┆ Current max DPD_1   ┆ Lifetime max DPD_1  ┆ Lifetime max DPD on_1 ┆ Total Overdue_1     ┆ Principal Overdue_1 ┆ EMI Overdue_1       ┆ POS_1               │
│ ---     ┆ ---         ┆ ---    ┆ ---      ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                